In [1]:
import itertools
import matplotlib as mpl
import numpy as np
import os
import tensorflow as tf
import tensorflow.contrib.slim as slim
import time
import seaborn as sns

from matplotlib import pyplot as plt
from scipy.misc import imsave
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

sns.set_style('whitegrid')

distributions = tf.contrib.distributions

flags = tf.app.flags
flags.DEFINE_string('data_dir', '/tmp/dat/', 'Directory for data')
flags.DEFINE_string('logdir', '/tmp/log/', 'Directory for logs')


In [2]:

# For making plots:
flags.DEFINE_integer('latent_dim', 2, 'Latent dimensionality of model')
flags.DEFINE_integer('batch_size', 64, 'Minibatch size')
flags.DEFINE_integer('n_samples', 10, 'Number of samples to save')
flags.DEFINE_integer('print_every', 10, 'Print every n iterations')
flags.DEFINE_integer('hidden_size', 200, 'Hidden size for neural networks')
flags.DEFINE_integer('n_iterations', 100000, 'number of iterations')

In [3]:

FLAGS = flags.FLAGS


def inference_network(x, latent_dim, hidden_size):
  """Construct an inference network parametrizing a Gaussian.
  Args:
    x: A batch of MNIST digits.
    latent_dim: The latent dimensionality.
    hidden_size: The size of the neural net hidden layers.
  Returns:
    mu: Mean parameters for the variational family Normal
    sigma: Standard deviation parameters for the variational family Normal
  """
  with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
    net = slim.flatten(x)
    net = slim.fully_connected(net, hidden_size)
    net = slim.fully_connected(net, hidden_size)
    gaussian_params = slim.fully_connected(
        net, latent_dim * 2, activation_fn=None)
  # The mean parameter is unconstrained
  mu = gaussian_params[:, :latent_dim]
  # The standard deviation must be positive. Parametrize with a softplus
  sigma = tf.nn.softplus(gaussian_params[:, latent_dim:])
  return mu, sigma


In [4]:

def generative_network(z, hidden_size):
  """Build a generative network parametrizing the likelihood of the data
  Args:
    z: Samples of latent variables
    hidden_size: Size of the hidden state of the neural net
  Returns:
    bernoulli_logits: logits for the Bernoulli likelihood of the data
  """
  with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
    net = slim.fully_connected(z, hidden_size)
    net = slim.fully_connected(net, hidden_size)
    bernoulli_logits = slim.fully_connected(net, 784, activation_fn=None)
    bernoulli_logits = tf.reshape(bernoulli_logits, [-1, 28, 28, 1])
  return bernoulli_logits


def train():
  # Train a Variational Autoencoder on MNIST

  # Input placeholders
  with tf.name_scope('data'):
    x = tf.placeholder(tf.float32, [None, 28, 28, 1])
    tf.summary.image('data', x)

  with tf.variable_scope('variational'):
    q_mu, q_sigma = inference_network(x=x,
                                      latent_dim=FLAGS.latent_dim,
                                      hidden_size=FLAGS.hidden_size)
    # The variational distribution is a Normal with mean and standard
    # deviation given by the inference network
    q_z = distributions.Normal(loc=q_mu, scale=q_sigma)
    assert q_z.reparameterization_type == distributions.FULLY_REPARAMETERIZED

  with tf.variable_scope('model'):
    # The likelihood is Bernoulli-distributed with logits given by the
    # generative network
    p_x_given_z_logits = generative_network(z=q_z.sample(),
                                            hidden_size=FLAGS.hidden_size)
    p_x_given_z = distributions.Bernoulli(logits=p_x_given_z_logits)
    posterior_predictive_samples = p_x_given_z.sample()
    tf.summary.image('posterior_predictive',
                     tf.cast(posterior_predictive_samples, tf.float32))

  # Take samples from the prior
  with tf.variable_scope('model', reuse=True):
    p_z = distributions.Normal(loc=np.zeros(FLAGS.latent_dim, dtype=np.float32),
                               scale=np.ones(FLAGS.latent_dim, dtype=np.float32))
    p_z_sample = p_z.sample(FLAGS.n_samples)
    p_x_given_z_logits = generative_network(z=p_z_sample,
                                            hidden_size=FLAGS.hidden_size)
    prior_predictive = distributions.Bernoulli(logits=p_x_given_z_logits)
    prior_predictive_samples = prior_predictive.sample()
    tf.summary.image('prior_predictive',
                     tf.cast(prior_predictive_samples, tf.float32))

  # Take samples from the prior with a placeholder
  with tf.variable_scope('model', reuse=True):
    z_input = tf.placeholder(tf.float32, [None, FLAGS.latent_dim])
    p_x_given_z_logits = generative_network(z=z_input,
                                            hidden_size=FLAGS.hidden_size)
    prior_predictive_inp = distributions.Bernoulli(logits=p_x_given_z_logits)
    prior_predictive_inp_sample = prior_predictive_inp.sample()

  # Build the evidence lower bound (ELBO) or the negative loss
  kl = tf.reduce_sum(distributions.kl(q_z, p_z), 1)
  expected_log_likelihood = tf.reduce_sum(p_x_given_z.log_prob(x),
                                          [1, 2, 3])

  elbo = tf.reduce_sum(expected_log_likelihood - kl, 0)

  optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

  train_op = optimizer.minimize(-elbo)

  # Merge all the summaries
  summary_op = tf.summary.merge_all()

  init_op = tf.global_variables_initializer()

  # Run training
  sess = tf.InteractiveSession()
  sess.run(init_op)

  mnist = read_data_sets(FLAGS.data_dir, one_hot=True)

  print('Saving TensorBoard summaries and images to: %s' % FLAGS.logdir)
  train_writer = tf.summary.FileWriter(FLAGS.logdir, sess.graph)

  # Get fixed MNIST digits for plotting posterior means during training
  np_x_fixed, np_y = mnist.test.next_batch(5000)
  np_x_fixed = np_x_fixed.reshape(5000, 28, 28, 1)
  np_x_fixed = (np_x_fixed > 0.5).astype(np.float32)

  for i in range(FLAGS.n_iterations):
    # Re-binarize the data at every batch; this improves results
    np_x, _ = mnist.train.next_batch(FLAGS.batch_size)
    np_x = np_x.reshape(FLAGS.batch_size, 28, 28, 1)
    np_x = (np_x > 0.5).astype(np.float32)
    sess.run(train_op, {x: np_x})

    # Print progress and save samples every so often
    t0 = time.time()
    if i % FLAGS.print_every == 0:
      np_elbo, summary_str = sess.run([elbo, summary_op], {x: np_x})
      train_writer.add_summary(summary_str, i)
      print('Iteration: {0:d} ELBO: {1:.3f} Examples/s: {2:.3e}'.format(
          i,
          np_elbo / FLAGS.batch_size,
          FLAGS.batch_size * FLAGS.print_every / (time.time() - t0)))
      t0 = time.time()

      # Save samples
      np_posterior_samples, np_prior_samples = sess.run(
          [posterior_predictive_samples, prior_predictive_samples], {x: np_x})
      for k in range(FLAGS.n_samples):
        f_name = os.path.join(
            FLAGS.logdir, 'iter_%d_posterior_predictive_%d_data.jpg' % (i, k))
        imsave(f_name, np_x[k, :, :, 0])
        f_name = os.path.join(
            FLAGS.logdir, 'iter_%d_posterior_predictive_%d_sample.jpg' % (i, k))
        imsave(f_name, np_posterior_samples[k, :, :, 0])
        f_name = os.path.join(
            FLAGS.logdir, 'iter_%d_prior_predictive_%d.jpg' % (i, k))
        imsave(f_name, np_prior_samples[k, :, :, 0])

      # Plot the posterior predictive space
      if FLAGS.latent_dim == 2:
        np_q_mu = sess.run(q_mu, {x: np_x_fixed})
        cmap = mpl.colors.ListedColormap(sns.color_palette("husl"))
        f, ax = plt.subplots(1, figsize=(6 * 1.1618, 6))
        im = ax.scatter(np_q_mu[:, 0], np_q_mu[:, 1], c=np.argmax(np_y, 1), cmap=cmap,
                        alpha=0.7)
        ax.set_xlabel('First dimension of sampled latent variable $z_1$')
        ax.set_ylabel('Second dimension of sampled latent variable mean $z_2$')
        ax.set_xlim([-10., 10.])
        ax.set_ylim([-10., 10.])
        f.colorbar(im, ax=ax, label='Digit class')
        plt.tight_layout()
        plt.savefig(os.path.join(FLAGS.logdir,
                                 'posterior_predictive_map_frame_%d.jpg' % i))
        plt.close()

        nx = ny = 20
        x_values = np.linspace(-3, 3, nx)
        y_values = np.linspace(-3, 3, ny)
        canvas = np.empty((28 * ny, 28 * nx))
        for ii, yi in enumerate(x_values):
          for j, xi in enumerate(y_values):
            np_z = np.array([[xi, yi]])
            x_mean = sess.run(prior_predictive_inp_sample, {z_input: np_z})
            canvas[(nx - ii - 1) * 28:(nx - ii) * 28, j *
                   28:(j + 1) * 28] = x_mean[0].reshape(28, 28)
        imsave(os.path.join(FLAGS.logdir,
                            'prior_predictive_map_frame_%d.jpg' % i), canvas)
        # plt.figure(figsize=(8, 10))
        # Xi, Yi = np.meshgrid(x_values, y_values)
        # plt.imshow(canvas, origin="upper")
        # plt.tight_layout()
        # plt.savefig()

  # Make the gifs (Do this by hand)
  # convert -delay 15 -loop 0 posterior_predictive_map_frame*png posterior_predictive.gif
  # if FLAGS.latent_dim == 2:
  #   os.system(
  #       'convert -delay 15 -loop 0 {0}/posterior_predictive_map_frame*jpg {0}/posterior_predictive.gif'
  #       .format(FLAGS.logdir))
  #   os.system(
  #       'convert -delay 15 -loop 0 {0}/prior_predictive_map_frame*jpg {0}/prior_predictive.gif'
  #       .format(FLAGS.logdir))
# 

In [5]:

def main(_):
  if tf.gfile.Exists(FLAGS.logdir):
    tf.gfile.DeleteRecursively(FLAGS.logdir)
  tf.gfile.MakeDirs(FLAGS.logdir)
  train()


if __name__ == '__main__':
  tf.app.run()

Extracting /tmp/dat/train-images-idx3-ubyte.gz
Extracting /tmp/dat/train-labels-idx1-ubyte.gz
Extracting /tmp/dat/t10k-images-idx3-ubyte.gz
Extracting /tmp/dat/t10k-labels-idx1-ubyte.gz
Saving TensorBoard summaries and images to: /tmp/log/
Iteration: 0 ELBO: -540.898 Examples/s: 2.312e+04


/anaconda/envs/py27.13/lib/python2.7/site-packages/ipykernel_launcher.py:119: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/anaconda/envs/py27.13/lib/python2.7/site-packages/ipykernel_launcher.py:122: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/anaconda/envs/py27.13/lib/python2.7/site-packages/ipykernel_launcher.py:125: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/anaconda/envs/py27.13/lib/python2.7/site-packages/ipykernel_launcher.py:155: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration: 10 ELBO: -252.895 Examples/s: 1.761e+05
Iteration: 20 ELBO: -214.260 Examples/s: 2.085e+05
Iteration: 30 ELBO: -209.449 Examples/s: 2.098e+05
Iteration: 40 ELBO: -203.628 Examples/s: 2.011e+05
Iteration: 50 ELBO: -207.312 Examples/s: 2.270e+05
Iteration: 60 ELBO: -212.680 Examples/s: 2.202e+05
Iteration: 70 ELBO: -204.979 Examples/s: 1.955e+05
Iteration: 80 ELBO: -192.478 Examples/s: 1.885e+05
Iteration: 90 ELBO: -206.828 Examples/s: 2.309e+05
Iteration: 100 ELBO: -192.465 Examples/s: 2.060e+05
Iteration: 110 ELBO: -205.134 Examples/s: 2.030e+05
Iteration: 120 ELBO: -198.399 Examples/s: 2.275e+05
Iteration: 130 ELBO: -191.473 Examples/s: 2.444e+05
Iteration: 140 ELBO: -200.246 Examples/s: 2.272e+05
Iteration: 150 ELBO: -187.156 Examples/s: 2.229e+05
Iteration: 160 ELBO: -184.352 Examples/s: 2.246e+05
Iteration: 170 ELBO: -196.242 Examples/s: 2.332e+05
Iteration: 180 ELBO: -180.881 Examples/s: 2.639e+05
Iteration: 190 ELBO: -180.209 Examples/s: 2.132e+05
Iteration: 200 ELBO: 

Iteration: 1580 ELBO: -164.592 Examples/s: 2.643e+05
Iteration: 1590 ELBO: -157.298 Examples/s: 2.440e+05
Iteration: 1600 ELBO: -155.641 Examples/s: 2.274e+05
Iteration: 1610 ELBO: -152.216 Examples/s: 2.484e+05
Iteration: 1620 ELBO: -145.812 Examples/s: 2.437e+05
Iteration: 1630 ELBO: -148.385 Examples/s: 2.428e+05
Iteration: 1640 ELBO: -147.918 Examples/s: 2.401e+05
Iteration: 1650 ELBO: -155.474 Examples/s: 2.393e+05
Iteration: 1660 ELBO: -156.013 Examples/s: 2.464e+05
Iteration: 1670 ELBO: -157.429 Examples/s: 2.369e+05
Iteration: 1680 ELBO: -155.320 Examples/s: 2.414e+05
Iteration: 1690 ELBO: -163.733 Examples/s: 2.336e+05
Iteration: 1700 ELBO: -159.255 Examples/s: 2.429e+05
Iteration: 1710 ELBO: -153.366 Examples/s: 2.534e+05
Iteration: 1720 ELBO: -147.788 Examples/s: 2.324e+05
Iteration: 1730 ELBO: -162.741 Examples/s: 2.519e+05
Iteration: 1740 ELBO: -145.112 Examples/s: 2.474e+05
Iteration: 1750 ELBO: -146.403 Examples/s: 2.501e+05
Iteration: 1760 ELBO: -148.679 Examples/s: 2.3

Iteration: 3130 ELBO: -127.332 Examples/s: 2.316e+05
Iteration: 3140 ELBO: -152.565 Examples/s: 2.454e+05
Iteration: 3150 ELBO: -151.689 Examples/s: 2.181e+05
Iteration: 3160 ELBO: -138.049 Examples/s: 2.303e+05
Iteration: 3170 ELBO: -135.990 Examples/s: 2.393e+05
Iteration: 3180 ELBO: -158.357 Examples/s: 2.264e+05
Iteration: 3190 ELBO: -152.617 Examples/s: 2.160e+05
Iteration: 3200 ELBO: -148.984 Examples/s: 2.316e+05
Iteration: 3210 ELBO: -156.535 Examples/s: 2.056e+05
Iteration: 3220 ELBO: -146.624 Examples/s: 2.424e+05
Iteration: 3230 ELBO: -155.973 Examples/s: 2.266e+05
Iteration: 3240 ELBO: -147.042 Examples/s: 2.032e+05
Iteration: 3250 ELBO: -134.910 Examples/s: 2.269e+05
Iteration: 3260 ELBO: -138.718 Examples/s: 2.208e+05
Iteration: 3270 ELBO: -145.491 Examples/s: 2.411e+05
Iteration: 3280 ELBO: -146.442 Examples/s: 2.044e+05
Iteration: 3290 ELBO: -146.006 Examples/s: 2.429e+05
Iteration: 3300 ELBO: -141.912 Examples/s: 2.251e+05
Iteration: 3310 ELBO: -145.083 Examples/s: 2.3

...

Iteration: 95980 ELBO: -140.708 Examples/s: 2.282e+05
Iteration: 95990 ELBO: -147.978 Examples/s: 2.325e+05
Iteration: 96000 ELBO: -125.830 Examples/s: 2.091e+05
Iteration: 96010 ELBO: -135.945 Examples/s: 2.113e+05
Iteration: 96020 ELBO: -132.318 Examples/s: 2.059e+05
Iteration: 96030 ELBO: -126.793 Examples/s: 2.249e+05
Iteration: 96040 ELBO: -137.196 Examples/s: 2.133e+05
Iteration: 96050 ELBO: -125.643 Examples/s: 2.141e+05
Iteration: 96060 ELBO: -127.136 Examples/s: 2.088e+05
Iteration: 96070 ELBO: -135.783 Examples/s: 1.911e+05
Iteration: 96080 ELBO: -132.225 Examples/s: 2.293e+05
Iteration: 96090 ELBO: -132.258 Examples/s: 2.091e+05
Iteration: 96100 ELBO: -126.695 Examples/s: 2.134e+05
Iteration: 96110 ELBO: -126.888 Examples/s: 1.958e+05
Iteration: 96120 ELBO: -127.571 Examples/s: 2.173e+05
Iteration: 96130 ELBO: -121.339 Examples/s: 2.379e+05
Iteration: 96140 ELBO: -127.260 Examples/s: 2.041e+05
Iteration: 96150 ELBO: -139.680 Examples/s: 2.079e+05
Iteration: 96160 ELBO: -119.

Iteration: 97500 ELBO: -135.240 Examples/s: 2.028e+05
Iteration: 97510 ELBO: -129.856 Examples/s: 1.887e+05
Iteration: 97520 ELBO: -128.261 Examples/s: 1.946e+05
Iteration: 97530 ELBO: -133.102 Examples/s: 2.202e+05
Iteration: 97540 ELBO: -124.693 Examples/s: 2.263e+05
Iteration: 97550 ELBO: -130.158 Examples/s: 2.237e+05
Iteration: 97560 ELBO: -137.182 Examples/s: 2.257e+05
Iteration: 97570 ELBO: -124.947 Examples/s: 2.365e+05
Iteration: 97580 ELBO: -130.034 Examples/s: 1.888e+05
Iteration: 97590 ELBO: -129.884 Examples/s: 2.189e+05
Iteration: 97600 ELBO: -126.091 Examples/s: 2.233e+05
Iteration: 97610 ELBO: -127.712 Examples/s: 2.048e+05
Iteration: 97620 ELBO: -122.761 Examples/s: 2.103e+05
Iteration: 97630 ELBO: -130.951 Examples/s: 2.053e+05
Iteration: 97640 ELBO: -128.148 Examples/s: 2.285e+05
Iteration: 97650 ELBO: -137.154 Examples/s: 2.057e+05
Iteration: 97660 ELBO: -125.706 Examples/s: 2.021e+05
Iteration: 97670 ELBO: -124.476 Examples/s: 2.156e+05
Iteration: 97680 ELBO: -136.

Iteration: 99020 ELBO: -130.088 Examples/s: 2.343e+05
Iteration: 99030 ELBO: -127.992 Examples/s: 2.061e+05
Iteration: 99040 ELBO: -142.052 Examples/s: 2.304e+05
Iteration: 99050 ELBO: -136.696 Examples/s: 2.311e+05
Iteration: 99060 ELBO: -117.601 Examples/s: 2.219e+05
Iteration: 99070 ELBO: -120.050 Examples/s: 2.250e+05
Iteration: 99080 ELBO: -132.091 Examples/s: 2.196e+05
Iteration: 99090 ELBO: -128.386 Examples/s: 2.375e+05
Iteration: 99100 ELBO: -124.344 Examples/s: 2.160e+05
Iteration: 99110 ELBO: -127.933 Examples/s: 2.445e+05
Iteration: 99120 ELBO: -131.342 Examples/s: 2.400e+05
Iteration: 99130 ELBO: -125.957 Examples/s: 2.098e+05
Iteration: 99140 ELBO: -128.357 Examples/s: 2.177e+05
Iteration: 99150 ELBO: -119.339 Examples/s: 2.418e+05
Iteration: 99160 ELBO: -131.537 Examples/s: 2.417e+05
Iteration: 99170 ELBO: -115.014 Examples/s: 2.233e+05
Iteration: 99180 ELBO: -133.318 Examples/s: 2.071e+05
Iteration: 99190 ELBO: -138.972 Examples/s: 2.276e+05
Iteration: 99200 ELBO: -145.

SystemExit: 

//anaconda/envs/py27.13/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
